# Experiment 10: Alpha-Sweep - Mapping the Levy-Gaussian Transition

**Phase 4 - Testing the Critical Alpha Hypothesis**

## Background

Experiment 9 revealed that **noise regime asymmetry** is the dominant source of hysteresis:
- Cascade phase: Levy noise (alpha=1.5) with fat-tailed extreme events
- Recovery phase: Gaussian noise (alpha=2.0) with bounded perturbations
- Result: ~0% recovery because Gaussian noise lacks the extreme jumps needed to escape tipped state

## Hypothesis

There exists a **critical alpha value** (approximately 1.7) that separates:
- **Levy regime** (alpha < 1.7): Heavy-tailed noise enables barrier crossing and recovery
- **Gaussian regime** (alpha >= 1.7): Bounded noise traps system in tipped state

## This Experiment

Systematically sweep recovery alpha from 1.1 to 2.0 to:
1. Map recovery fraction as a function of alpha
2. Identify the critical alpha threshold
3. Characterize whether the transition is sharp or gradual
4. Measure thermodynamic signatures (entropy production) across regimes

| Parameter | Value |
|-----------|-------|
| Alpha values | 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0 |
| Cascade alpha (fixed) | 1.5 |
| Ensemble runs | 30 per alpha |
| Total simulations | 300 |

## 1. Setup and Imports

In [ ]:
import sys
sys.path.insert(0, '/opt/research-local/src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import time
from pathlib import Path
from netCDF4 import Dataset
from dask.distributed import as_completed

# Core energy-constrained module
from energy_constrained import (
    EnergyConstrainedNetwork,
    EnergyConstrainedCusp,
    GradientDrivenCoupling,
    run_two_phase_experiment,
    EnergyAnalyzer,
    get_dask_client
)

print("Imports successful!")
print(f"NumPy version: {np.__version__}")

In [ ]:
# Connect to Dask cluster
client = get_dask_client()
print(f"Connected to: {client.scheduler_info()['address']}")
print(f"Workers: {len(client.scheduler_info()['workers'])}")
print(f"Total threads: {sum(w['nthreads'] for w in client.scheduler_info()['workers'].values())}")

## 2. Load Amazon Moisture Recycling Data

In [ ]:
DATA_PATH = Path('/opt/research-local/data/amazon/amazon_adaptation_model/average_network/era5_new_network_data')

def load_amazon_data(year=2003, months=[7, 8, 9]):
    """Load and average Amazon moisture recycling data for specified months."""
    all_rain = []
    all_evap = []
    all_network = []
    
    for month in months:
        file_path = DATA_PATH / f'1deg_{year}_{month:02d}.nc'
        if file_path.exists():
            with Dataset(file_path, 'r') as ds:
                all_rain.append(ds.variables['rain'][:])
                all_evap.append(ds.variables['evap'][:])
                all_network.append(ds.variables['network'][:])
    
    return {
        'rain': np.mean(all_rain, axis=0),
        'evap': np.mean(all_evap, axis=0),
        'network': np.mean(all_network, axis=0),
        'n_cells': len(all_rain[0])
    }

# Load 2003 (normal year) data
amazon_data = load_amazon_data(year=2003)
print(f"Loaded Amazon data: {amazon_data['n_cells']} cells")
print(f"Network shape: {amazon_data['network'].shape}")
print(f"Rain range: {amazon_data['rain'].min():.1f} - {amazon_data['rain'].max():.1f} mm")

## 3. Experiment Configuration

In [ ]:
# Experiment 10: Alpha-Sweep Configuration
# Tests how recovery changes as a function of noise stability parameter alpha

SWEEP_CONFIG = {
    # Alpha sweep parameters
    'alpha_values': np.round(np.arange(1.1, 2.05, 0.1), 1),  # [1.1, 1.2, ..., 2.0]
    'n_runs_per_alpha': 30,
    
    # Network parameters
    'n_cells': 50,
    'min_flow': 1.0,
    'barrier_height': 0.2,  # Option D from Exp 9
    
    # Two-phase simulation parameters
    'cascade_duration': 200,
    'recovery_duration': 800,
    'dt': 0.5,
    'cascade_sigma': 0.06,
    'cascade_alpha': 1.5,  # Fixed Levy for cascade
    'recovery_sigma': 0.04,  # Option D from Exp 9
    # recovery_alpha varies in sweep
    
    # Seeds
    'base_seed': 42,
}

total_sims = len(SWEEP_CONFIG['alpha_values']) * SWEEP_CONFIG['n_runs_per_alpha']

print("=" * 60)
print("EXPERIMENT 10: ALPHA-SWEEP CONFIGURATION")
print("=" * 60)
print(f"Alpha values to test: {SWEEP_CONFIG['alpha_values']}")
print(f"Runs per alpha: {SWEEP_CONFIG['n_runs_per_alpha']}")
print(f"Total simulations: {total_sims}")
print(f"\nFixed parameters:")
print(f"  Cascade alpha: {SWEEP_CONFIG['cascade_alpha']} (Levy)")
print(f"  Cascade sigma: {SWEEP_CONFIG['cascade_sigma']}")
print(f"  Recovery sigma: {SWEEP_CONFIG['recovery_sigma']}")
print(f"  Barrier height: {SWEEP_CONFIG['barrier_height']}")
print(f"\nHypothesis: Critical alpha ~ 1.7 separates recovery-capable from recovery-incapable regimes")

## 4. Network Creation

In [ ]:
def create_sweep_network(data, config, seed=42):
    """
    Create a 50-cell network for alpha sweep experiments.
    Uses symmetric EnergyConstrainedCusp elements with Option D barrier height.
    """
    np.random.seed(seed)
    
    network_matrix = data['network']
    n_cells = config['n_cells']
    min_flow = config['min_flow']
    barrier_height = config['barrier_height']
    
    # Select top cells by total connectivity
    total_flow = network_matrix.sum(axis=0) + network_matrix.sum(axis=1)
    top_indices = np.argsort(total_flow)[-n_cells:]
    
    # Create network
    net = EnergyConstrainedNetwork()
    
    # Add cusp elements
    for i, idx in enumerate(top_indices):
        element = EnergyConstrainedCusp(
            a=-1.0, b=1.0, c=0.0, x_0=0.0,
            barrier_height=barrier_height,
            dissipation_rate=0.1
        )
        net.add_element(f'cell_{i}', element)
    
    # Add couplings based on moisture flow
    n_edges = 0
    for i, idx_i in enumerate(top_indices):
        for j, idx_j in enumerate(top_indices):
            if i != j:
                flow = network_matrix[idx_i, idx_j]
                if flow > min_flow:
                    coupling = GradientDrivenCoupling(
                        conductivity=flow / 100.0,
                        state_coupling=0.1
                    )
                    net.add_coupling(f'cell_{i}', f'cell_{j}', coupling)
                    n_edges += 1
    
    return net, top_indices

# Create the network (will be serialized for workers)
network, selected_cells = create_sweep_network(amazon_data, SWEEP_CONFIG)
print(f"Created network: {network.n_elements} nodes, {network.number_of_edges()} edges")
print(f"Barrier height: {SWEEP_CONFIG['barrier_height']}")

## 5. Worker Function Definition

In [ ]:
def run_single_alpha_experiment(network_bytes, recovery_alpha, config, seed):
    """
    Worker function for a single alpha sweep experiment.
    
    Parameters
    ----------
    network_bytes : bytes
        Pickled network object (for Dask serialization)
    recovery_alpha : float
        Levy stability parameter for recovery phase
    config : dict
        Experiment configuration
    seed : int
        Random seed for this run
    
    Returns
    -------
    dict
        Results dictionary for DataFrame aggregation
    """
    import sys
    import numpy as np
    import pickle
    
    # Add path for k3s workers
    if '/opt/research-local/src' not in sys.path:
        sys.path.insert(0, '/opt/research-local/src')
    
    from energy_constrained.solvers import run_two_phase_experiment
    from energy_constrained.analysis import EnergyAnalyzer
    
    # Reconstruct network
    network = pickle.loads(network_bytes)
    
    # Set seed
    np.random.seed(seed)
    
    # Run two-phase experiment with this recovery alpha
    result = run_two_phase_experiment(
        network=network,
        cascade_duration=config['cascade_duration'],
        recovery_duration=config['recovery_duration'],
        dt=config['dt'],
        cascade_sigma=config['cascade_sigma'],
        cascade_alpha=config['cascade_alpha'],
        recovery_sigma=config['recovery_sigma'],
        recovery_alpha=recovery_alpha,  # THE SWEEP VARIABLE
        seed=seed
    )
    
    # Compute additional metrics
    # Time in tipped state
    pct_time_tipped = np.mean(result.x_full > 0) * 100
    
    # Count transitions (tip and recovery events)
    n_cells = result.x_full.shape[1]
    n_tip_events = 0
    n_recover_events = 0
    
    for j in range(n_cells):
        x_traj = result.x_full[:, j]
        # Find crossings of x=0
        signs = np.sign(x_traj)
        sign_changes = np.diff(signs)
        n_tip_events += np.sum(sign_changes > 0)  # - to + = tip
        n_recover_events += np.sum(sign_changes < 0)  # + to - = recover
    
    return {
        'recovery_alpha': recovery_alpha,
        'seed': seed,
        'recovery_fraction': result.metrics['recovery_fraction'],
        'pct_tipped_cascade': result.metrics['pct_tipped_at_cascade_end'],
        'n_permanent_tips': result.metrics['n_permanent_tips'],
        'final_pct_tipped': result.metrics['final_pct_tipped'],
        'pct_time_tipped': pct_time_tipped,
        'n_tip_events': n_tip_events,
        'n_recover_events': n_recover_events,
        'mean_recovery_time': result.metrics['mean_recovery_time'],
    }

print("Worker function defined.")

## 6. Run Alpha Sweep Experiment

In [ ]:
# Serialize network and SCATTER to workers (optimization)
# This sends data ONCE to all workers instead of with every task
network_bytes = pickle.dumps(network)
network_future = client.scatter(network_bytes, broadcast=True)
print(f"Network scattered to {len(client.scheduler_info()['workers'])} workers ({len(network_bytes) / 1024:.1f} KB)")

# Submit all tasks
print("\n" + "=" * 60)
print("EXPERIMENT 10: Starting Alpha-Sweep")
print("=" * 60)
start_time = time.time()

futures = []
task_info = []  # Track (alpha, run_idx) for each future

for i, alpha in enumerate(SWEEP_CONFIG['alpha_values']):
    for run_idx in range(SWEEP_CONFIG['n_runs_per_alpha']):
        seed = SWEEP_CONFIG['base_seed'] + i * 1000 + run_idx
        
        future = client.submit(
            run_single_alpha_experiment,
            network_bytes=network_future,  # Use scattered reference
            recovery_alpha=float(alpha),
            config=SWEEP_CONFIG,
            seed=seed,
            key=f"alpha_{alpha:.1f}_run_{run_idx}",
            pure=False  # Stochastic simulations
        )
        futures.append(future)
        task_info.append((alpha, run_idx))

print(f"Submitted {len(futures)} tasks to Dask cluster")
print(f"Alpha values: {list(SWEEP_CONFIG['alpha_values'])}")
print(f"Runs per alpha: {SWEEP_CONFIG['n_runs_per_alpha']}")

# Collect results with progress tracking
all_results = []
completed_by_alpha = {float(a): 0 for a in SWEEP_CONFIG['alpha_values']}

print("\nProgress:")
for future in as_completed(futures):
    result = future.result()
    all_results.append(result)
    
    # Track progress by alpha
    alpha = result['recovery_alpha']
    completed_by_alpha[alpha] += 1
    
    # Print progress every 30 completions (one full alpha)
    total_completed = len(all_results)
    if total_completed % 30 == 0:
        print(f"  Completed {total_completed}/{len(futures)} simulations...")

elapsed = time.time() - start_time
print(f"\nTotal runtime: {elapsed:.1f} seconds ({elapsed/60:.1f} minutes)")
print(f"Average per simulation: {elapsed/len(futures):.2f} seconds")

## 7. Results Aggregation

In [ ]:
# Convert to DataFrame
df = pd.DataFrame(all_results)
print(f"Results shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")

# Sort by alpha for consistent ordering
df = df.sort_values(['recovery_alpha', 'seed']).reset_index(drop=True)

df.head(10)

In [ ]:
# Summary statistics by alpha
summary = df.groupby('recovery_alpha').agg({
    'recovery_fraction': ['mean', 'std', 'min', 'max'],
    'pct_tipped_cascade': ['mean', 'std'],
    'n_permanent_tips': ['mean', 'std'],
    'final_pct_tipped': ['mean', 'std'],
    'pct_time_tipped': ['mean', 'std'],
    'n_tip_events': ['mean', 'std'],
    'n_recover_events': ['mean', 'std'],
}).round(3)

summary.columns = ['_'.join(col) for col in summary.columns]
summary = summary.reset_index()

print("=" * 80)
print("ALPHA-SWEEP SUMMARY: Recovery by Noise Stability Parameter")
print("=" * 80)
print(summary[['recovery_alpha', 'recovery_fraction_mean', 'recovery_fraction_std', 
               'n_permanent_tips_mean', 'n_recover_events_mean']].to_string(index=False))

## 8. Primary Visualization - Transition Curves

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

alpha_vals = summary['recovery_alpha'].values

# Panel 1: Recovery Fraction vs Alpha (KEY METRIC)
ax = axes[0, 0]
means = summary['recovery_fraction_mean'].values
stds = summary['recovery_fraction_std'].values
ax.errorbar(alpha_vals, means, yerr=stds, marker='o', capsize=4, linewidth=2, markersize=8, color='blue')
ax.axhline(0.5, color='gray', linestyle='--', alpha=0.5, label='50% recovery')
ax.axvline(1.7, color='red', linestyle=':', alpha=0.7, linewidth=2, label='Hypothesized critical alpha')
ax.fill_between([1.6, 1.8], 0, 1, alpha=0.1, color='red')
ax.set_xlabel('Recovery Alpha', fontsize=12)
ax.set_ylabel('Recovery Fraction', fontsize=12)
ax.set_title('Recovery Success vs Noise Stability Parameter', fontsize=14)
ax.set_xlim(1.0, 2.1)
ax.set_ylim(0, 1)
ax.legend()
ax.grid(True, alpha=0.3)

# Panel 2: Permanent Tips vs Alpha
ax = axes[0, 1]
means = summary['n_permanent_tips_mean'].values
stds = summary['n_permanent_tips_std'].values
ax.errorbar(alpha_vals, means, yerr=stds, marker='s', capsize=4, linewidth=2, markersize=8, color='red')
ax.axvline(1.7, color='red', linestyle=':', alpha=0.7, linewidth=2)
ax.set_xlabel('Recovery Alpha', fontsize=12)
ax.set_ylabel('Permanent Tips (out of 50)', fontsize=12)
ax.set_title('Irreversibility vs Noise Stability Parameter', fontsize=14)
ax.set_xlim(1.0, 2.1)
ax.grid(True, alpha=0.3)

# Panel 3: Recovery Events vs Alpha
ax = axes[1, 0]
means = summary['n_recover_events_mean'].values
stds = summary['n_recover_events_std'].values
ax.errorbar(alpha_vals, means, yerr=stds, marker='^', capsize=4, linewidth=2, markersize=8, color='green')
ax.axvline(1.7, color='red', linestyle=':', alpha=0.7, linewidth=2)
ax.set_xlabel('Recovery Alpha', fontsize=12)
ax.set_ylabel('Number of Recovery Events', fontsize=12)
ax.set_title('Recovery Activity vs Noise Stability Parameter', fontsize=14)
ax.set_xlim(1.0, 2.1)
ax.grid(True, alpha=0.3)

# Panel 4: % Time Tipped vs Alpha
ax = axes[1, 1]
means = summary['pct_time_tipped_mean'].values
stds = summary['pct_time_tipped_std'].values
ax.errorbar(alpha_vals, means, yerr=stds, marker='d', capsize=4, linewidth=2, markersize=8, color='purple')
ax.axhline(50, color='gray', linestyle='--', alpha=0.5, label='50% time')
ax.axvline(1.7, color='red', linestyle=':', alpha=0.7, linewidth=2)
ax.set_xlabel('Recovery Alpha', fontsize=12)
ax.set_ylabel('% Time in Tipped State', fontsize=12)
ax.set_title('Time in Tipped State vs Noise Stability Parameter', fontsize=14)
ax.set_xlim(1.0, 2.1)
ax.set_ylim(0, 100)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('/workspace/data/exp10_transition_curves.png', dpi=150, bbox_inches='tight')
plt.show()
print("\nPlot saved to /workspace/data/exp10_transition_curves.png")

## 9. Critical Alpha Identification

In [ ]:
def find_critical_alpha(df, metric='recovery_fraction', threshold=0.1, method='interpolation'):
    """
    Find alpha where metric crosses threshold.
    
    For recovery_fraction: finds alpha where recovery drops below threshold
    (i.e., transition from recovery-capable to recovery-incapable)
    """
    grouped = df.groupby('recovery_alpha')[metric].agg(['mean', 'std', 'count'])
    
    alphas = grouped.index.values
    means = grouped['mean'].values
    
    # Find where mean crosses threshold (going from high to low as alpha increases)
    for i in range(len(alphas) - 1):
        if means[i] >= threshold and means[i+1] < threshold:
            # Crossing between i and i+1
            slope = (means[i+1] - means[i]) / (alphas[i+1] - alphas[i])
            if slope != 0:
                critical = alphas[i] + (threshold - means[i]) / slope
            else:
                critical = (alphas[i] + alphas[i+1]) / 2
            
            return {
                'critical_alpha': critical,
                'lower_bound': alphas[i],
                'upper_bound': alphas[i+1],
                'lower_value': means[i],
                'upper_value': means[i+1],
                'method': 'linear_interpolation',
                'threshold': threshold
            }
    
    # Check if always above or below threshold
    if np.all(means >= threshold):
        return {'critical_alpha': None, 'interpretation': f'All values above {threshold}'}
    elif np.all(means < threshold):
        return {'critical_alpha': None, 'interpretation': f'All values below {threshold}'}
    
    return {'critical_alpha': None, 'interpretation': 'No clear crossing found'}


# Find critical alpha at different thresholds
print("=" * 60)
print("CRITICAL ALPHA ANALYSIS")
print("=" * 60)

thresholds = [0.30, 0.20, 0.10, 0.05]

for thresh in thresholds:
    result = find_critical_alpha(df, threshold=thresh)
    if result['critical_alpha'] is not None:
        print(f"\nThreshold = {thresh*100:.0f}% recovery:")
        print(f"  Critical alpha: {result['critical_alpha']:.2f}")
        print(f"  Transition region: [{result['lower_bound']:.1f}, {result['upper_bound']:.1f}]")
        print(f"  Values: {result['lower_value']:.3f} -> {result['upper_value']:.3f}")
    else:
        print(f"\nThreshold = {thresh*100:.0f}%: {result.get('interpretation', 'Unknown')}")

In [ ]:
def bootstrap_critical_alpha(df, metric='recovery_fraction', threshold=0.1, n_bootstrap=1000):
    """Estimate critical alpha uncertainty via bootstrap resampling."""
    critical_alphas = []
    
    for _ in range(n_bootstrap):
        # Resample within each alpha group
        resampled_dfs = []
        for alpha in df['recovery_alpha'].unique():
            group = df[df['recovery_alpha'] == alpha]
            resampled = group.sample(n=len(group), replace=True)
            resampled_dfs.append(resampled)
        resampled_df = pd.concat(resampled_dfs, ignore_index=True)
        
        result = find_critical_alpha(resampled_df, metric, threshold)
        if result['critical_alpha'] is not None:
            critical_alphas.append(result['critical_alpha'])
    
    if len(critical_alphas) > 0:
        return {
            'mean': np.mean(critical_alphas),
            'std': np.std(critical_alphas),
            'ci_95': (np.percentile(critical_alphas, 2.5), np.percentile(critical_alphas, 97.5)),
            'n_valid': len(critical_alphas),
            'n_bootstrap': n_bootstrap
        }
    else:
        return {'mean': None, 'interpretation': 'Could not find critical alpha in bootstrap samples'}


# Bootstrap CI for 10% threshold
print("\n" + "=" * 60)
print("BOOTSTRAP CONFIDENCE INTERVALS")
print("=" * 60)

for thresh in [0.20, 0.10, 0.05]:
    print(f"\nBootstrapping critical alpha for {thresh*100:.0f}% threshold...")
    bs_result = bootstrap_critical_alpha(df, threshold=thresh, n_bootstrap=1000)
    
    if bs_result['mean'] is not None:
        print(f"  Critical alpha: {bs_result['mean']:.3f} +/- {bs_result['std']:.3f}")
        print(f"  95% CI: [{bs_result['ci_95'][0]:.3f}, {bs_result['ci_95'][1]:.3f}]")
        print(f"  Valid samples: {bs_result['n_valid']}/{bs_result['n_bootstrap']}")
    else:
        print(f"  {bs_result.get('interpretation', 'Unknown')}")

## 10. Distribution Visualization

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

alpha_vals = sorted(df['recovery_alpha'].unique())

# Panel 1: Box plots
ax = axes[0]
data_by_alpha = [df[df['recovery_alpha'] == a]['recovery_fraction'].values for a in alpha_vals]
bp = ax.boxplot(data_by_alpha, labels=[f'{a:.1f}' for a in alpha_vals], patch_artist=True)

# Color by regime
colors = []
for a in alpha_vals:
    if a < 1.5:
        colors.append('green')  # Strong Levy
    elif a < 1.8:
        colors.append('gold')  # Transition
    else:
        colors.append('red')  # Near-Gaussian

for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.6)

ax.axhline(0.5, color='gray', linestyle='--', alpha=0.5)
ax.set_xlabel('Recovery Alpha', fontsize=12)
ax.set_ylabel('Recovery Fraction', fontsize=12)
ax.set_title('Recovery Fraction Distribution by Alpha', fontsize=14)
ax.grid(True, alpha=0.3, axis='y')

# Add legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='green', alpha=0.6, label='Levy regime (alpha < 1.5)'),
    Patch(facecolor='gold', alpha=0.6, label='Transition (1.5 <= alpha < 1.8)'),
    Patch(facecolor='red', alpha=0.6, label='Gaussian regime (alpha >= 1.8)')
]
ax.legend(handles=legend_elements, loc='upper right')

# Panel 2: Violin plots
ax = axes[1]
parts = ax.violinplot(data_by_alpha, positions=range(len(alpha_vals)),
                      showmeans=True, showmedians=True)

for i, pc in enumerate(parts['bodies']):
    pc.set_facecolor(colors[i])
    pc.set_alpha(0.6)

ax.set_xticks(range(len(alpha_vals)))
ax.set_xticklabels([f'{a:.1f}' for a in alpha_vals])
ax.axhline(0.5, color='gray', linestyle='--', alpha=0.5)
ax.set_xlabel('Recovery Alpha', fontsize=12)
ax.set_ylabel('Recovery Fraction', fontsize=12)
ax.set_title('Recovery Fraction Violin Plot by Alpha', fontsize=14)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('/workspace/data/exp10_distributions.png', dpi=150, bbox_inches='tight')
plt.show()
print("\nPlot saved to /workspace/data/exp10_distributions.png")

## 11. Regime Classification

In [ ]:
# Classify each alpha into regime based on recovery fraction
def classify_regime(recovery_fraction):
    if recovery_fraction >= 0.30:
        return 'Recovery-capable'
    elif recovery_fraction >= 0.10:
        return 'Transition'
    else:
        return 'Trapped'

regime_table = []
for alpha in sorted(df['recovery_alpha'].unique()):
    subset = df[df['recovery_alpha'] == alpha]
    mean_recovery = subset['recovery_fraction'].mean()
    std_recovery = subset['recovery_fraction'].std()
    mean_events = subset['n_recover_events'].mean()
    regime = classify_regime(mean_recovery)
    
    regime_table.append({
        'Alpha': alpha,
        'Recovery Mean': f"{mean_recovery:.3f}",
        'Recovery Std': f"{std_recovery:.3f}",
        'Recovery Events': f"{mean_events:.1f}",
        'Regime': regime
    })

regime_df = pd.DataFrame(regime_table)

print("=" * 70)
print("REGIME CLASSIFICATION TABLE")
print("=" * 70)
print(regime_df.to_string(index=False))

# Identify transition region
transition_alphas = regime_df[regime_df['Regime'] == 'Transition']['Alpha'].values
if len(transition_alphas) > 0:
    print(f"\nTransition region: alpha in [{transition_alphas.min()}, {transition_alphas.max()}]")
    
# Compare to hypothesis
print(f"\nHypothesis: Critical alpha ~ 1.7")
if len(transition_alphas) > 0:
    mid_transition = (transition_alphas.min() + transition_alphas.max()) / 2
    print(f"Observed midpoint: {mid_transition:.2f}")

## 12. Key Findings Summary

In [ ]:
print("\n" + "=" * 70)
print("EXPERIMENT 10: KEY FINDINGS")
print("=" * 70)

# Find the actual critical alpha
result_10pct = find_critical_alpha(df, threshold=0.10)
bs_10pct = bootstrap_critical_alpha(df, threshold=0.10, n_bootstrap=500)

# Format critical alpha string safely
if result_10pct.get('critical_alpha') is not None:
    critical_str = f"{result_10pct['critical_alpha']:.2f}"
    hypothesis_result = 'SUPPORTED' if 1.5 < result_10pct['critical_alpha'] < 1.9 else 'NEEDS REVIEW'
else:
    critical_str = f"N/A ({result_10pct.get('interpretation', 'unknown')})"
    hypothesis_result = 'CANNOT EVALUATE - no critical alpha found'

# Format bootstrap CI string safely
if bs_10pct.get('ci_95') is not None:
    ci_str = f"[{bs_10pct['ci_95'][0]:.2f}, {bs_10pct['ci_95'][1]:.2f}]"
else:
    ci_str = f"N/A ({bs_10pct.get('interpretation', 'unknown')})"

# Format transition region strings safely
if len(transition_alphas) > 0:
    levy_bound = f"{transition_alphas.min():.1f}"
    trans_min = f"{transition_alphas.min():.1f}"
    trans_max = f"{transition_alphas.max():.1f}"
    gauss_bound = f"{transition_alphas.max():.1f}"
    trans_char = 'Sharp transition' if len(transition_alphas) <= 2 else 'Gradual transition'
    trans_span = len(transition_alphas)
else:
    levy_bound = trans_min = trans_max = gauss_bound = "?"
    trans_char = "Unknown"
    trans_span = 0

print(f"""
1. CRITICAL ALPHA IDENTIFICATION:
   Threshold: 10% recovery fraction
   Critical alpha: {critical_str}
   Bootstrap 95% CI: {ci_str}
   
2. HYPOTHESIS TEST:
   Hypothesized critical alpha: 1.7
   Observed: {hypothesis_result}

3. REGIME BOUNDARIES:
   Levy regime (recovery-capable):   alpha < {levy_bound}
   Transition zone:                  {trans_min} <= alpha <= {trans_max}
   Gaussian regime (trapped):        alpha > {gauss_bound}

4. TRANSITION CHARACTER:
   {trans_char} spanning {trans_span} alpha values

5. RECOVERY STATISTICS BY REGIME:
""")

# Print recovery stats by regime
for regime in ['Recovery-capable', 'Transition', 'Trapped']:
    alphas_in_regime = regime_df[regime_df['Regime'] == regime]['Alpha'].values
    if len(alphas_in_regime) > 0:
        subset = df[df['recovery_alpha'].isin(alphas_in_regime)]
        print(f"   {regime}:")
        print(f"     Alpha range: [{alphas_in_regime.min():.1f}, {alphas_in_regime.max():.1f}]")
        print(f"     Mean recovery: {subset['recovery_fraction'].mean():.3f}")
        print(f"     Mean recovery events: {subset['n_recover_events'].mean():.1f}")
    else:
        print(f"   {regime}: No alpha values in this regime")

## 13. Save Results

In [ ]:
# Save full results
df.to_csv('/workspace/data/experiment10_alpha_sweep_results.csv', index=False)
print(f"Full results saved to /workspace/data/experiment10_alpha_sweep_results.csv")

# Save summary
summary.to_csv('/workspace/data/experiment10_summary.csv', index=False)
print(f"Summary saved to /workspace/data/experiment10_summary.csv")

# Save regime classification
regime_df.to_csv('/workspace/data/experiment10_regimes.csv', index=False)
print(f"Regime classification saved to /workspace/data/experiment10_regimes.csv")

In [ ]:
print("\n" + "=" * 70)
print("EXPERIMENT 10 COMPLETE")
print("=" * 70)

print(f"""
CONFIGURATION:
- Alpha values tested: {len(SWEEP_CONFIG['alpha_values'])} ({SWEEP_CONFIG['alpha_values'].min():.1f} to {SWEEP_CONFIG['alpha_values'].max():.1f})
- Runs per alpha: {SWEEP_CONFIG['n_runs_per_alpha']}
- Total simulations: {len(df)}
- Runtime: {elapsed:.1f} seconds ({elapsed/60:.1f} minutes)

FILES GENERATED:
- /workspace/data/experiment10_alpha_sweep_results.csv
- /workspace/data/experiment10_summary.csv
- /workspace/data/experiment10_regimes.csv
- /workspace/data/exp10_transition_curves.png
- /workspace/data/exp10_distributions.png

NEXT STEPS:
1. Update docs/phase4_results.md with these findings
2. If critical alpha differs significantly from 1.7, investigate why
3. Consider Experiment 11 (Keystone Connections) or 12 (Publication Statistics)
""")